In [1]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
from datetime import datetime
import re
from sqlalchemy import create_engine, exc

import pymysql
from pymysql.err import IntegrityError, InternalError

In [5]:
load_dotenv()

app_id = os.getenv("TDX_APP_ID")
app_key = os.getenv("TDX_APP_KEY")

auth_url="https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"
url = "https://tdx.transportdata.tw/api/basic/v2/Rail/TRA/LiveTrainDelay?$top=30&$format=JSON"

class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        content_type = 'application/x-www-form-urlencoded'
        grant_type = 'client_credentials'

        return{
            'content-type' : content_type,
            'grant_type' : grant_type,
            'client_id' : self.app_id,
            'client_secret' : self.app_key
        }

class data():

    def __init__(self, app_id, app_key, auth_response):
        self.app_id = app_id
        self.app_key = app_key
        self.auth_response = auth_response

    def get_data_header(self):
        if self.auth_response=="":
            access_token = ""
        else:
            auth_JSON = json.loads(self.auth_response.text)
            access_token = auth_JSON.get('access_token')

        return{
            'authorization': 'Bearer ' + access_token,
            'Accept-Encoding': 'gzip'
        }



In [20]:
url = "https://tdx.transportdata.tw/api/basic/v2/Rail/Metro/StationExit/TRTC?format=JSON"
a = Auth(app_id, app_key)
auth_response = requests.post(auth_url, a.get_auth_header())
d = data(app_id, app_key, auth_response)
data_response = requests.get(url, headers=d.get_data_header())


In [26]:
def E_mrt_station():
    load_dotenv()

    app_id = os.getenv("TDX_APP_ID")
    app_key = os.getenv("TDX_APP_KEY")

    auth_url="https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"
    url =   'https://tdx.transportdata.tw/api/basic/v2/Rail/Metro/Station/TRTC?format=JSON'
    a = Auth(app_id, app_key)
    auth_response = requests.post(auth_url, a.get_auth_header())
    d = data(app_id, app_key, auth_response)
    response = requests.get(url, headers=d.get_data_header())
    return(pd.json_normalize(json.loads(response.text),max_level=1))

In [37]:
df = E_mrt_station()
df

,StationUID,StationID,StationAddress,BikeAllowOnHoliday,SrcUpdateTime,UpdateTime,VersionID,LocationCity,LocationCityCode,LocationTown,LocationTownCode,StationName.Zh_tw,StationName.En,StationPosition.PositionLon,StationPosition.PositionLat,StationPosition.GeoHash
0,TRTC-BL01,BL01,236040新北市土城區中央路4段51之6號B3,True,2023-10-12T00:00:00+08:00,2023-10-17T00:00:00+08:00,6,新北市,NWT,土城區,65000130,頂埔,Dingpu,121.418744,24.959351,wsqmfzwqv
1,TRTC-BL02,BL02,236036新北市土城區中央路3段105號B1,True,2023-10-12T00:00:00+08:00,2023-10-17T00:00:00+08:00,6,新北市,NWT,土城區,65000130,永寧,Yongning,121.436130,24.966820,wsqq53546
2,TRTC-BL03,BL03,236017新北市土城區金城路1段105號B1,True,2023-10-12T00:00:00+08:00,2023-10-17T00:00:00+08:00,6,新北市,NWT,土城區,65000130,土城,Tucheng,121.444320,24.973130,wsqq5d1p1
3,TRTC-BL04,BL04,236023新北市土城區海山路39號B2,True,2023-10-12T00:00:00+08:00,2023-10-17T00:00:00+08:00,6,新北市,NWT,土城區,65000130,海山,Haishan,121.448787,24.985348,wsqq5skq1
4,TRTC-BL05,BL05,220056新北市板橋區南雅南路2段17號B1,True,2023-10-12T00:00:00+08:00,2023-10-17T00:00:00+08:00,6,新北市,NWT,板橋區,65000010,亞東醫院,Far Eastern Hospital,121.452465,24.998280,wsqq5wycx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,TRTC-R25,R25,112021臺北市北投區大度路3段296巷51號,True,2023-10-12T00:00:00+08:00,2023-10-17T00:00:00+08:00,6,臺北市,TPE,北投區,63000120,關渡,Guandu,121.467100,25.125630,wsqqujcxk
117,TRTC-R26,R26,251033新北市淡水區民權路50號,True,2023-10-12T00:00:00+08:00,2023-10-17T00:00:00+08:00,6,新北市,NWT,淡水區,65000100,竹圍,Zhuwei,121.459550,25.136900,wsqr5bh1h
118,TRTC-R27,R27,251033新北市淡水區中正東路2段68號,True,2023-10-12T00:00:00+08:00,2023-10-17T00:00:00+08:00,6,新北市,NWT,淡水區,65000100,紅樹林,Hongshulin,121.458800,25.153990,wsqr5g5se
119,TRTC-R28,R28,251018新北市淡水區中正路1號,False,2023-10-12T00:00:00+08:00,2023-10-17T00:00:00+08:00,6,新北市,NWT,淡水區,65000100,淡水,Tamsui,121.445805,25.167745,wsqr5tdhs


In [38]:
pattern = re.compile(r"^[A-Z]+")
def pattern_match_station_line_type(x):
    if pattern.findall(x):
        return (pattern.findall(x)[0])
    else:
        return ("")
df["line_type"] = df["StationID"].apply(pattern_match_station_line_type)

In [40]:
df = df.loc[:,['StationID','line_type','StationName.Zh_tw', 'StationName.En', 'StationAddress', 
               'StationPosition.PositionLat', 'StationPosition.PositionLon' ,
               'LocationCityCode', 'LocationTown','BikeAllowOnHoliday', 'UpdateTime' ]]

In [43]:
df.rename(columns={
    "StationID":"mrt_station_id",
    "StationName.Zh_tw":"mrt_station_name",
    "StationName.En":"station_en",
    "StationAddress":"station_address",
    "StationPosition.PositionLat":"lat",
    "StationPosition.PositionLon":"lon",
    "LocationCityCode":"city_code",
    "LocationTown":"district",
    "BikeAllowOnHoliday":"bike_allow_on_holiday",
    "UpdateTime":"update_time"
},inplace=True)

In [44]:
df

,mrt_station_id,line_type,mrt_station_name,station_en,station_address,lat,lon,city_code,district,bike_allow_on_holiday,update_time
0,BL01,BL,頂埔,Dingpu,236040新北市土城區中央路4段51之6號B3,24.959351,121.418744,NWT,土城區,True,2023-10-17T00:00:00+08:00
1,BL02,BL,永寧,Yongning,236036新北市土城區中央路3段105號B1,24.966820,121.436130,NWT,土城區,True,2023-10-17T00:00:00+08:00
2,BL03,BL,土城,Tucheng,236017新北市土城區金城路1段105號B1,24.973130,121.444320,NWT,土城區,True,2023-10-17T00:00:00+08:00
3,BL04,BL,海山,Haishan,236023新北市土城區海山路39號B2,24.985348,121.448787,NWT,土城區,True,2023-10-17T00:00:00+08:00
4,BL05,BL,亞東醫院,Far Eastern Hospital,220056新北市板橋區南雅南路2段17號B1,24.998280,121.452465,NWT,板橋區,True,2023-10-17T00:00:00+08:00
...,...,...,...,...,...,...,...,...,...,...,...
116,R25,R,關渡,Guandu,112021臺北市北投區大度路3段296巷51號,25.125630,121.467100,TPE,北投區,True,2023-10-17T00:00:00+08:00
117,R26,R,竹圍,Zhuwei,251033新北市淡水區民權路50號,25.136900,121.459550,NWT,淡水區,True,2023-10-17T00:00:00+08:00
118,R27,R,紅樹林,Hongshulin,251033新北市淡水區中正東路2段68號,25.153990,121.458800,NWT,淡水區,True,2023-10-17T00:00:00+08:00
119,R28,R,淡水,Tamsui,251018新北市淡水區中正路1號,25.167745,121.445805,NWT,淡水區,False,2023-10-17T00:00:00+08:00


In [33]:
from mrt_realtime_crowded_BL import E_mrt_crowded_BL





In [34]:
E_mrt_crowded_BL()

E_mrt_crowded_BL finished


,TrainNumber,CID,StationID,StationName,CN1,Car1,Car2,Car3,Car4,Car5,Car6,UpdateTime
0,220,上行,BL01,頂埔,115/116,1,1,1,1,1,1,2024/05/06 10:25:28
1,219,上行,BL04,海山,169/170,1,1,1,1,1,1,2024/05/06 10:26:06
2,209,上行,BL06,府中,139/140,1,2,2,2,1,1,2024/05/06 10:26:47
3,218,上行,BL09,江子翠,149/150,2,2,3,3,3,2,2024/05/06 10:27:28
4,208,上行,BL10,龍山寺,161/162,1,2,2,2,3,2,2024/05/06 10:26:19
5,217,上行,BL11,西門,113/114,2,2,2,2,2,2,2024/05/06 10:24:07
6,207,上行,BL14,忠孝新生,153/154,2,3,3,3,3,2,2024/05/06 10:26:06
7,216,上行,BL17,國父紀念館,133/134,2,2,2,2,2,1,2024/05/06 10:26:09
8,206,上行,BL20,後山埤,135/136,1,1,1,1,1,1,2024/05/06 10:26:55
9,215,上行,BL22,南港,101/102,1,1,1,1,1,1,2024/05/06 10:26:35


In [ ]:
def L_mrt_station(df: pd.DataFrame):
    username_sql = os.getenv("ANDY_USERNAME_SQL")
    password_sql = os.getenv("ANDY_PASSWORD_SQL")
    # server = "host.docker.internal:3306"  #docker用
    server = "localhost:3306"
    db_name = "group2_db"
    with create_engine(f"mysql+pymysql://{username_sql}:{password_sql}@{server}/{db_name}",).connect() as conn:
            df.to_sql(
                name="mrt_station",
                con=conn,
                if_exists="replace",
                index=False
            )

    print("L_mrt_station finished")
    return ("L_mrt_station finished")

In [45]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
from datetime import datetime
import re
from sqlalchemy import create_engine, exc
from zoneinfo import ZoneInfo
from io import StringIO


# mrt_usage_history
# get csv download of every month's data
# each url can get one month's data
def E_mrt_usage_history_csvfilelist():
    url = "https://data.taipei/api/dataset/63f31c7e-7fc3-418b-bd82-b95158755b4d/resource/eb481f58-1238-4cff-8caa-fa7bb20cb4f4/download"
    response = requests.get(url=url)
    response_list = response.text.split("\r")

    col_name = response_list[0].split(",")
    url_df = pd.concat([pd.DataFrame([response_list[i].split(
        ",")[1:]], columns=col_name[1:]) for i in range(1, len(response_list))], axis=0)
    url_df.reset_index(drop=True, inplace=True)
    print("E_mrt_usage_history_csvfilelist finished")
    return (url_df)

In [46]:
url_list = E_mrt_usage_history_csvfilelist()

E_mrt_usage_history_csvfilelist finished


In [52]:
url = url_list.loc[0,"URL"]

In [53]:
url

'http://tcgmetro.blob.core.windows.net/stationod/%E8%87%BA%E5%8C%97%E6%8D%B7%E9%81%8B%E6%AF%8F%E6%97%A5%E5%88%86%E6%99%82%E5%90%84%E7%AB%99OD%E6%B5%81%E9%87%8F%E7%B5%B1%E8%A8%88%E8%B3%87%E6%96%99_201701.csv'

In [54]:
response = requests.get(url=url)
StringIO_df = StringIO(response.content.decode("utf-8-sig"))
df = pd.read_csv(StringIO_df)
pattern = re.compile(r"[A-Za-z]+")
df["進站"] = df["進站"].str.replace(pattern, "", regex=True)
df["出站"] = df["出站"].str.replace(pattern, "", regex=True)
df

,日期,時段,進站,出站,人次
0,2017-01-01,0,松山機場,松山機場,0
1,2017-01-01,0,松山機場,中山國中,0
2,2017-01-01,0,松山機場,南京復興,0
3,2017-01-01,0,松山機場,忠孝復興,0
4,2017-01-01,0,松山機場,大安,0
...,...,...,...,...,...
7581595,2017-01-31,23,丹鳳,徐匯中學,2
7581596,2017-01-31,23,丹鳳,三和國中,1
7581597,2017-01-31,23,丹鳳,三重國小,4
7581598,2017-01-31,23,丹鳳,迴龍,0


In [56]:
df.rename(columns={
    "日期":"date",
    "時段":"hour",
    "進站":"mrt_station_name_enter",
    "出站":"mrt_station_name_exit",	
    "人次":"visitors_num"
},inplace=True)

In [57]:
df

,date,hour,mrt_station_name_enter,mrt_station_name_exit,visitors_num
0,2017-01-01,0,松山機場,松山機場,0
1,2017-01-01,0,松山機場,中山國中,0
2,2017-01-01,0,松山機場,南京復興,0
3,2017-01-01,0,松山機場,忠孝復興,0
4,2017-01-01,0,松山機場,大安,0
...,...,...,...,...,...
7581595,2017-01-31,23,丹鳳,徐匯中學,2
7581596,2017-01-31,23,丹鳳,三和國中,1
7581597,2017-01-31,23,丹鳳,三重國小,4
7581598,2017-01-31,23,丹鳳,迴龍,0
